# Imports

In [1]:
#Always make all imports in the first cell of the notebook, run them all once.
import numpy as np
from Preproccessing import preproccessing
from FeatureExtraction import extract_features
from IO import ReadDataSet
from Phase1 import Phase1
from sklearn.model_selection import train_test_split
import time
from sklearn.metrics import f1_score, accuracy_score
import cv2
import glob


%load_ext autoreload
%autoreload 2
%matplotlib inline

# Read Dataset

In [ ]:
'''
1.a) READ THE DATASET
'''
x,y = ReadDataSet()

'''
 1.b) Splitting the data 
        Training Set : 70%
        Test Set : 15 %
'''
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=42)

c:\Users\aliaa\Documents\Pattern\IO.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x= np.array(x)


# PHASE-1
Training Phase

In [ ]:
model = Phase1(x_train,y_train)

# PHASE-2
Testing Phase

In [ ]:
# # read test data from file
# x_test = []
# for filename in sorted(glob.glob('./test/*.*')):
#     img = cv2.imread(filename) 
#     x_test.append(img)

# y_test = [ 1, 7 ,2 ,3]
# print(len(x_test))
# print(y_test)

In [ ]:
y_pred = np.zeros(len(x_test))
runningTime = np.zeros(len(x_test))

# 1) Loop on test data
total_start = time.time()
for i in range(0,len(x_test)):
    # start timer
    start = time.time()
    # 2) Preprocess and extract features from the test data
    x_test[i] = preproccessing(x_test[i].astype('uint8'))
    x_test[i] = extract_features(x_test[i])
    y_pred[i] = model.predict([x_test[i]])
    # stop timer
    runningTime[i] = time.time()-start

total_running = time.time() - total_start


In [ ]:
# calculate model accuracy and F1Score
print('runningTime in sec =', total_running)
print('Accuracy = ' , (sum(y_pred == y_test)/len(y_test))*100)
print('F1Score = ',  f1_score(y_test, y_pred, average='weighted') *100)

runningTime in sec = 20.102794647216797
Accuracy =  98.81305637982196
F1Score =  98.81366952068548
